# Import libraries

In [1]:
import os
import copy
from tqdm.notebook import tqdm
import pprint
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import torch
from src.constants import DATA_PATH, MASTER_THESIS_DIR, TRAINING_CONFIG_PATH, MODEL_CONFIG_PATH
from src.data_loader.data_set import Data_Set
from src.experiments.utils import get_experiement_args, process_experiment_args
from src.models.baseline_model import BaselineModel
from src.utils import get_console_logger, read_json
from src.visualization.visualize import plot_hand
from torch.utils.data import DataLoader
from torchvision import transforms
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
from src.data_loader.utils import convert_2_5D_to_3D, convert_to_2_5D
from src.experiments.evaluation_utils import calculate_epe_statistics, evaluate,get_predictions_and_ground_truth, cal_auc_joints, get_pck_curves

# Load configuration

In [23]:
model_config = edict(read_json(MODEL_CONFIG_PATH))
joints_mapping = {v: k for k, v in read_json("/local/home/adahiya/Documents/master_thesis/src/data_loader/joint_mapping.json")["ait"].items()}
model_config.gpu =True
train_config =  edict(read_json(TRAINING_CONFIG_PATH))
train_config.crop =True
train_config.rotate=True
print(model_config)

{'alpha': 5, 'gpu': True, 'resnet_trainable': True, 'learning_rate': 0.0001, 'scheduler': {'choice': 'cosine_annealing', 'cosine_annealing': {'T_max': 300}, 'reduce_on_plateau': {'mode': 'min', 'factor': 0.5, 'patience': 4, 'verbose': True}}}


# Load data

In [24]:
train_data = Data_Set(
    config=train_config,
    transforms=transforms.Compose(
        [transforms.ToTensor()]
    ),
    train_set=True,
)
val_data = copy.copy(train_data)
val_data.is_training(False)

# Load model

In [25]:
model = BaselineModel(config=model_config)
checkpoint = torch.load("/local/home/adahiya/Documents/master_thesis/data/models/master-thesis/8364a50940f44dc3963173d0f7b3c68a/checkpoints/epoch=284.ckpt"
)
model.load_state_dict(checkpoint["state_dict"])
model.eval()
device = torch.device("cuda")
model.to(device)
print()

# calculate Results

In [26]:
results = evaluate(model, val_data, num_workers=8,batch_size=32)
pprint.pprint(results)


407it [00:03, 115.76it/s]
100%|██████████| 13024/13024 [00:03<00:00, 4070.23it/s]


{'Mean_EPE_2D': tensor(4.9733),
 'Mean_EPE_3D': tensor(0.2035),
 'Mean_EPE_3D_R': tensor(0.0974),
 'Mean_EPE_3D_R_v_3D': tensor(0.1967),
 'Median_EPE_2D': tensor(3.4725),
 'Median_EPE_3D': tensor(0.1666),
 'Median_EPE_3D_R': tensor(0.0348),
 'Median_EPE_3D_R_V_3D': tensor(0.1611)}


In [16]:
results = evaluate(model, val_data, num_workers=8,batch_size=32)
pprint.pprint(results)
# without rotation

407it [00:03, 123.92it/s]
100%|██████████| 13024/13024 [00:03<00:00, 4070.15it/s]


{'Mean_EPE_2D': tensor(4.9900),
 'Mean_EPE_3D': tensor(0.1960),
 'Mean_EPE_3D_R': tensor(0.0951),
 'Mean_EPE_3D_R_v_3D': tensor(0.1832),
 'Median_EPE_2D': tensor(3.4940),
 'Median_EPE_3D': tensor(0.1498),
 'Median_EPE_3D_R': tensor(0.0349),
 'Median_EPE_3D_R_V_3D': tensor(0.1437)}


In [18]:
pred_dict = get_predictions_and_ground_truth(model.to(device), val_data,num_workers=8,batch_size=32)
pred = pred_dict["predictions_3d"]
gt = pred_dict["ground_truth_3d"]
sts =calculate_epe_statistics(pred, gt, dim=3)
eucledian_dist = sts['eucledian_dist']
y,x = get_pck_curves(eucledian_dist, per_joint=True)
auc = cal_auc_joints(eucledian_dist, per_joint=True)

407it [00:03, 123.89it/s]
100%|██████████| 13024/13024 [00:03<00:00, 3947.95it/s]


#  Visualize sample predictions

In [20]:
model.to(torch.device('cpu'))
print()

In [21]:
@interact(idx=widgets.IntSlider(min=1, max=100, step=5, value=3),plot_gt= widgets.Checkbox(value=True, description='Ground truth'),
         plot_pred= widgets.Checkbox(value=True, description='Predicted_labels'))
def visualize1(idx, plot_gt, plot_pred):
    sample = val_data[idx]
    joints = sample["joints"]
    img = sample["image"]
    img_input = img.view(([1]+list(img.shape)))
    prediction = model(img_input).view(21,3).detach().numpy()
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    plt.imshow(transforms.ToPILImage()(img))
    if plot_gt:
       plot_hand(ax, joints)
    if plot_pred:
        plot_hand(ax, prediction,linestyle=':' )
    plt.show()

interactive(children=(IntSlider(value=3, description='idx', min=1, step=5), Checkbox(value=True, description='…

In [19]:
@interact(joint_id=widgets.IntSlider(min=0, max=20, step=1, value=3))

def visualize2(joint_id):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    plt.plot(x,y[joint_id])
    plt.title(f"PCK curve, AUC: {auc[joint_id]}")
    plt.plot(x,np.mean(y,axis=0), color='black',linestyle=":" ,alpha=0.5)
    plt.xlabel("error in mm")
    plt.ylabel("Ratio of points below the error")
    plt.legend([f"PCK curve for {joints_mapping[joint_id]}", "Average PCK curve"])
    plt.show()

interactive(children=(IntSlider(value=3, description='joint_id', max=20), Output()), _dom_classes=('widget-int…